<a href="https://colab.research.google.com/github/MissTiny/Graph_Mining_Node_Edge_K-Mean_Clustering/blob/main/Social_Circle_Discovery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing libraries

In [ ]:
pip install node2vec

Imports.

In [1]:
import json
import numpy as np
import networkx as nx
import requests
from node2vec import Node2Vec

Downloading and writing data into file.

In [2]:
url = 'https://snap.stanford.edu/data/soc-sign-Slashdot090221.txt.gz'
r = requests.get(url, allow_redirects=True)
data_file = open('soc-sign-Slashdot090221.txt.gz', 'wb').write(r.content)

Reading signed network data into directed graph using networkx.

In [3]:
signed_network = nx.read_weighted_edgelist('soc-sign-Slashdot090221.txt.gz', comments='#', create_using=nx.DiGraph(), nodetype = int)

Updating weights (-1 to 1 and 1 to 2) as node2vec does not work on negative weights

In [ ]:
for from_node,to_node, weight in signed_network.edges(data=True):
    if weight['weight'] == -1:
        weight['weight'] = 1
    elif weight['weight'] == 1:
        weight['weight'] = 2
    print(from_node,to_node, weight)

0 1 {'weight': 2}
0 2 {'weight': 2}
0 3 {'weight': 2}
0 4 {'weight': 2}
0 5 {'weight': 2}
0 6 {'weight': 2}
0 7 {'weight': 2}
0 8 {'weight': 2}
0 9 {'weight': 2}
0 10 {'weight': 2}
0 11 {'weight': 2}
0 12 {'weight': 2}
0 13 {'weight': 2}
0 14 {'weight': 2}
0 15 {'weight': 2}
0 16 {'weight': 2}
0 17 {'weight': 2}
0 18 {'weight': 2}
0 19 {'weight': 2}
0 20 {'weight': 2}
0 21 {'weight': 2}
0 22 {'weight': 2}
0 23 {'weight': 2}
0 24 {'weight': 2}
0 25 {'weight': 1}
0 26 {'weight': 1}
0 27 {'weight': 1}
0 28 {'weight': 1}
0 29 {'weight': 1}
1 0 {'weight': 2}
1 4 {'weight': 2}
1 9 {'weight': 2}
1 31 {'weight': 2}
1 41 {'weight': 2}
1 52 {'weight': 2}
1 98 {'weight': 2}
1 118 {'weight': 2}
1 170 {'weight': 2}
1 530 {'weight': 1}
1 613 {'weight': 2}
1 614 {'weight': 2}
1 615 {'weight': 2}
1 616 {'weight': 2}
1 617 {'weight': 2}
1 618 {'weight': 2}
1 619 {'weight': 2}
1 620 {'weight': 2}
1 621 {'weight': 2}
1 622 {'weight': 2}
1 623 {'weight': 2}
1 624 {'weight': 2}
1 625 {'weight': 2}
1 626 {'

8 941 {'weight': 1}
9 17 {'weight': 2}
9 47 {'weight': 2}
9 48 {'weight': 2}
9 49 {'weight': 2}
9 52 {'weight': 2}
9 55 {'weight': 2}
9 56 {'weight': 2}
9 58 {'weight': 2}
9 60 {'weight': 2}
9 80 {'weight': 2}
9 98 {'weight': 2}
9 118 {'weight': 2}
9 134 {'weight': 2}
9 140 {'weight': 2}
9 144 {'weight': 2}
9 196 {'weight': 2}
9 622 {'weight': 2}
9 632 {'weight': 2}
9 699 {'weight': 2}
9 820 {'weight': 2}
9 843 {'weight': 2}
9 942 {'weight': 2}
9 943 {'weight': 2}
9 944 {'weight': 2}
9 945 {'weight': 2}
9 946 {'weight': 2}
9 947 {'weight': 2}
9 948 {'weight': 2}
9 949 {'weight': 2}
9 950 {'weight': 2}
9 951 {'weight': 2}
9 952 {'weight': 2}
9 953 {'weight': 2}
9 954 {'weight': 2}
9 955 {'weight': 2}
9 956 {'weight': 2}
9 957 {'weight': 2}
9 958 {'weight': 2}
9 959 {'weight': 1}
10 0 {'weight': 2}
10 19 {'weight': 2}
10 960 {'weight': 2}
10 961 {'weight': 2}
10 962 {'weight': 2}
10 963 {'weight': 2}
10 964 {'weight': 2}
10 965 {'weight': 2}
10 966 {'weight': 2}
10 967 {'weight': 2}
10 9

14 1289 {'weight': 2}
14 1290 {'weight': 2}
14 1291 {'weight': 2}
14 1292 {'weight': 2}
14 1293 {'weight': 2}
14 1294 {'weight': 2}
14 1295 {'weight': 2}
14 1296 {'weight': 2}
14 1297 {'weight': 2}
14 1298 {'weight': 2}
14 1299 {'weight': 2}
14 1300 {'weight': 2}
14 1301 {'weight': 2}
14 1302 {'weight': 2}
14 1303 {'weight': 2}
14 1304 {'weight': 2}
14 1305 {'weight': 1}
14 1306 {'weight': 1}
14 1307 {'weight': 1}
14 1308 {'weight': 1}
14 1309 {'weight': 1}
14 1310 {'weight': 1}
14 1311 {'weight': 1}
14 1312 {'weight': 1}
14 1313 {'weight': 1}
14 1314 {'weight': 1}
15 757 {'weight': 2}
15 1315 {'weight': 2}
16 0 {'weight': 2}
16 58 {'weight': 2}
16 87 {'weight': 2}
16 100 {'weight': 2}
16 159 {'weight': 2}
16 245 {'weight': 2}
16 510 {'weight': 2}
16 530 {'weight': 2}
16 641 {'weight': 2}
16 658 {'weight': 2}
16 780 {'weight': 2}
16 866 {'weight': 2}
16 906 {'weight': 1}
16 911 {'weight': 2}
16 997 {'weight': 2}
16 1024 {'weight': 2}
16 1072 {'weight': 2}
16 1079 {'weight': 2}
16 1088 

530 552 {'weight': 2}
530 555 {'weight': 2}
530 629 {'weight': 2}
530 661 {'weight': 2}
530 662 {'weight': 2}
530 668 {'weight': 2}
530 801 {'weight': 2}
530 838 {'weight': 2}
530 871 {'weight': 2}
530 874 {'weight': 2}
530 876 {'weight': 2}
530 905 {'weight': 2}
530 907 {'weight': 2}
530 911 {'weight': 2}
530 923 {'weight': 2}
530 928 {'weight': 2}
530 937 {'weight': 2}
530 938 {'weight': 2}
530 949 {'weight': 2}
530 973 {'weight': 2}
530 995 {'weight': 2}
530 998 {'weight': 2}
530 1000 {'weight': 2}
530 1006 {'weight': 2}
530 1030 {'weight': 2}
530 1062 {'weight': 2}
530 1087 {'weight': 2}
530 1111 {'weight': 2}
530 1118 {'weight': 2}
530 1134 {'weight': 2}
530 1136 {'weight': 2}
530 1140 {'weight': 2}
530 1152 {'weight': 2}
530 1173 {'weight': 2}
530 1183 {'weight': 2}
530 1201 {'weight': 2}
530 1213 {'weight': 2}
530 1233 {'weight': 2}
530 1239 {'weight': 2}
530 1240 {'weight': 2}
530 1264 {'weight': 2}
530 1276 {'weight': 2}
530 1282 {'weight': 2}
530 1297 {'weight': 2}
530 1300 {

616 1 {'weight': 2}
616 3 {'weight': 2}
616 11 {'weight': 2}
616 52 {'weight': 2}
616 53 {'weight': 2}
616 59 {'weight': 1}
616 60 {'weight': 2}
616 66 {'weight': 2}
616 67 {'weight': 2}
616 70 {'weight': 2}
616 76 {'weight': 2}
616 79 {'weight': 2}
616 83 {'weight': 2}
616 89 {'weight': 2}
616 92 {'weight': 2}
616 94 {'weight': 2}
616 100 {'weight': 2}
616 168 {'weight': 2}
616 170 {'weight': 2}
616 216 {'weight': 2}
616 351 {'weight': 2}
616 621 {'weight': 2}
616 624 {'weight': 2}
616 640 {'weight': 2}
616 657 {'weight': 2}
616 686 {'weight': 2}
616 725 {'weight': 2}
616 813 {'weight': 2}
616 832 {'weight': 2}
616 843 {'weight': 2}
616 846 {'weight': 2}
616 933 {'weight': 2}
616 949 {'weight': 2}
616 974 {'weight': 2}
616 989 {'weight': 2}
616 993 {'weight': 2}
616 1018 {'weight': 2}
616 1028 {'weight': 2}
616 1034 {'weight': 2}
616 1035 {'weight': 2}
616 1042 {'weight': 2}
616 1055 {'weight': 2}
616 1063 {'weight': 2}
616 1069 {'weight': 2}
616 1087 {'weight': 2}
616 1093 {'weight':

624 1869 {'weight': 2}
624 1877 {'weight': 2}
624 1883 {'weight': 2}
624 1923 {'weight': 2}
624 1929 {'weight': 2}
624 1935 {'weight': 2}
624 1939 {'weight': 2}
624 1946 {'weight': 2}
624 2034 {'weight': 2}
624 2054 {'weight': 2}
624 2225 {'weight': 2}
624 2510 {'weight': 2}
624 2518 {'weight': 2}
624 2523 {'weight': 2}
624 2528 {'weight': 2}
624 2539 {'weight': 2}
624 2549 {'weight': 2}
624 2571 {'weight': 2}
624 2806 {'weight': 2}
624 2938 {'weight': 2}
624 2970 {'weight': 2}
624 3068 {'weight': 2}
624 3601 {'weight': 2}
624 3974 {'weight': 2}
624 4311 {'weight': 2}
624 4346 {'weight': 2}
624 4368 {'weight': 2}
624 4575 {'weight': 2}
624 4693 {'weight': 2}
624 4941 {'weight': 2}
624 5050 {'weight': 2}
624 5161 {'weight': 2}
624 5311 {'weight': 2}
624 5323 {'weight': 2}
624 6949 {'weight': 2}
624 7138 {'weight': 2}
624 7615 {'weight': 2}
624 7721 {'weight': 2}
624 7926 {'weight': 2}
624 8403 {'weight': 2}
624 8405 {'weight': 2}
624 8482 {'weight': 2}
624 8501 {'weight': 2}
624 8677 {'

627 10763 {'weight': 1}
627 10764 {'weight': 1}
627 10765 {'weight': 1}
627 10766 {'weight': 1}
627 10767 {'weight': 1}
628 33 {'weight': 2}
628 41 {'weight': 2}
628 733 {'weight': 2}
628 36099 {'weight': 2}
629 8 {'weight': 2}
629 80 {'weight': 2}
629 197 {'weight': 2}
629 1357 {'weight': 2}
629 1360 {'weight': 2}
629 2173 {'weight': 2}
629 2488 {'weight': 2}
629 2888 {'weight': 2}
629 2933 {'weight': 2}
629 3097 {'weight': 2}
629 3161 {'weight': 2}
629 3440 {'weight': 2}
629 3569 {'weight': 2}
629 3933 {'weight': 2}
629 4082 {'weight': 2}
629 4092 {'weight': 2}
629 5625 {'weight': 2}
629 5654 {'weight': 2}
629 5927 {'weight': 2}
629 7006 {'weight': 2}
629 7769 {'weight': 2}
629 9313 {'weight': 2}
629 10818 {'weight': 2}
629 11270 {'weight': 2}
629 18796 {'weight': 2}
629 19551 {'weight': 2}
629 27030 {'weight': 2}
629 36100 {'weight': 2}
629 36101 {'weight': 2}
629 36102 {'weight': 2}
629 36103 {'weight': 2}
630 30 {'weight': 2}
630 41 {'weight': 2}
630 47 {'weight': 2}
630 172 {'wei

641 7754 {'weight': 2}
641 7836 {'weight': 2}
641 7854 {'weight': 2}
641 8045 {'weight': 2}
641 8107 {'weight': 2}
641 8163 {'weight': 2}
641 8173 {'weight': 2}
641 8178 {'weight': 2}
641 8612 {'weight': 2}
641 8663 {'weight': 2}
641 8738 {'weight': 2}
641 8938 {'weight': 2}
641 9470 {'weight': 2}
641 9498 {'weight': 2}
641 9519 {'weight': 2}
641 9577 {'weight': 2}
641 9956 {'weight': 2}
641 9978 {'weight': 2}
641 10013 {'weight': 2}
641 10014 {'weight': 2}
641 10028 {'weight': 2}
641 10038 {'weight': 2}
641 10332 {'weight': 2}
641 10443 {'weight': 2}
641 10457 {'weight': 2}
641 10625 {'weight': 2}
641 11898 {'weight': 2}
641 12009 {'weight': 2}
641 12107 {'weight': 2}
641 12295 {'weight': 2}
641 12305 {'weight': 2}
641 13009 {'weight': 2}
641 13416 {'weight': 2}
641 13437 {'weight': 2}
641 13584 {'weight': 2}
641 13987 {'weight': 2}
641 14447 {'weight': 2}
641 14527 {'weight': 2}
641 14706 {'weight': 2}
641 14732 {'weight': 2}
641 14831 {'weight': 2}
641 15370 {'weight': 2}
641 15405 

Generating node features using node2vec.

In [ ]:
EMBEDDING_FILENAME = "embedded-soc-sign-slashdot"

# Precompute probabilities and generate walks
node2vec = Node2Vec(signed_network, dimensions=100)
# Embed nodes
model = node2vec.fit()# Save embeddings for later use
model.wv.save_word2vec_format(EMBEDDING_FILENAME)
# Save model for later use
EMBEDDING_MODEL_FILENAME = "embedded-soc-sign-slashdot-model"
model.save(EMBEDDING_MODEL_FILENAME)

# Embed edges using Hadamard method
from node2vec.edges import HadamardEmbedder

edges_embs = HadamardEmbedder(keyed_vectors=model.wv)

# Save edge embeddings for later use
EDGES_EMBEDDING_FILENAME = "edges-embedded-soc-sign-slashdot"
# Get all edges in a separate KeyedVectors instance - use with caution could be huge for big networks
edges_kv = edges_embs.as_keyed_vectors()
edges_kv.save_word2vec_format(EDGES_EMBEDDING_FILENAME)